In [1]:
import tensorflow as tf
shift_right_module = tf.load_op_library('./tensorflow/right_shift.so')
shift_left_module = tf.load_op_library('./tensorflow/left_shift.so')
import numpy as np
import pandas as pd
np.set_printoptions(formatter={'int':hex})

Sbox = tf.constant(np.array([
        0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
        0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
        0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
        0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
        0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
        0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
        0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
        0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
        0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
        0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
        0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
        0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
        0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
        0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
        0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
        0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16
        ]), dtype=tf.int32)

def hamming_weight(x):
    m1  = 0x55
    m2  = 0x33
    m4  = 0x0f
    B,C = 0,0
    y = tf.bitwise.bitwise_or(tf.bitwise.bitwise_and(tf.bitwise.bitwise_and(x, shift_left_module.left_shift(x, 1)),
                               shift_left_module.left_shift(m1, 1)), tf.bitwise.bitwise_and(m1, tf.bitwise.bitwise_xor(x, shift_right_module.right_shift(x, 1))))
    B = tf.bitwise.bitwise_and(y, m2)
    C = tf.bitwise.bitwise_and(shift_right_module.right_shift(y,  2), m2)
    y = tf.bitwise.bitwise_xor(B, tf.bitwise.bitwise_xor(C, shift_left_module.left_shift(tf.bitwise.bitwise_and(B, C), 1)))
    B = tf.bitwise.bitwise_xor(tf.bitwise.bitwise_and(y, m4 ), tf.bitwise.bitwise_and(shift_right_module.right_shift(y,  4), m4))
    C = shift_left_module.left_shift(tf.bitwise.bitwise_and(y, tf.bitwise.bitwise_and(shift_right_module.right_shift(y,  4), m4)), 1)
    y = tf.bitwise.bitwise_xor(B, tf.bitwise.bitwise_xor(C, shift_left_module.left_shift(tf.bitwise.bitwise_and(B, C), 1)))
    return y

def correlation(x_i, y_i):
    x, y = tf.to_double(x_i), tf.to_double(y_i)
    cov_xx = 1 / (tf.shape(x)[0] - 1) * tf.reduce_sum((x - tf.reduce_mean(x))**2)
    cov_yy = 1 / (tf.shape(x)[0] - 1) * tf.reduce_sum((y - tf.reduce_mean(y))**2)
    cov_xy = 1 / (tf.shape(x)[0] - 1) * tf.reduce_sum((x - tf.reduce_mean(x)) * (y - tf.reduce_mean(y)))

    return cov_xy/(tf.sqrt(cov_xx*cov_yy))

In [2]:
print("On importe les données de T...")
T_df= pd.read_csv("./data/T.csv", header=None,sep=",")
T = tf.transpose(tf.constant(T_df.as_matrix(),
                name='T', dtype=tf.int32))

print("On importe les données de M...")
M_df= pd.read_csv("./data/M.csv", header=None,sep=",")
M = tf.transpose(tf.constant(M_df.as_matrix(),
                name='M', dtype=tf.int32))

On importe les données de T...
On importe les données de M...


In [3]:
cles_hypothetiques = tf.constant(np.transpose(np.array([range(0,256)]*len(T_df))), name='cles_hypothetiques', dtype=tf.int32)
indice_octet = tf.placeholder(tf.int32)
vecteur_de_T = T[indice_octet]
cles_de_tour_ajoutees = tf.bitwise.bitwise_xor(cles_hypothetiques, vecteur_de_T)
modeles = tf.to_double(hamming_weight(tf.map_fn(lambda x: tf.map_fn(lambda y: Sbox[y], x), cles_de_tour_ajoutees)))
correlations = tf.to_double(M)
correlations = tf.abs(tf.map_fn(lambda x: tf.map_fn(lambda y: correlation(x,y),modeles)
                         ,correlations))

In [4]:
import time
t0 = time.time()
sess = tf.Session()

for i in range(0,16):
    print("Recherche de k{}...".format(i))
    corr = sess.run(correlations, feed_dict={indice_octet: i})
    ki = np.unravel_index(corr.argmax(), corr.shape) [1]
    print(corr)
    print("k{} = {}".format(i,ki))

print("Temps de traitement {} s".format(time.time()-t0))

Recherche de k0...
[[ 0.00666022  0.00970412  0.0058257  ...,  0.00089969  0.0046037
   0.01183688]
 [ 0.01757837  0.01562236  0.00542263 ...,  0.01459296  0.00918904
   0.01790135]
 [ 0.01820089  0.01376706  0.01176446 ...,  0.01149546  0.00313489
   0.00081479]
 ..., 
 [ 0.00597981  0.00175518  0.01175968 ...,  0.00243009  0.00020056
   0.00973869]
 [ 0.00078057  0.00582332  0.01345306 ...,  0.00383965  0.01285775
   0.0181017 ]
 [ 0.00751918  0.01275439  0.01739058 ...,  0.00049915  0.0119414
   0.02490445]]
k0 = 0
Recherche de k1...
[[ 0.00254873  0.01339081  0.00405153 ...,  0.00979914  0.0095307
   0.00563103]
 [ 0.01405741  0.01016571  0.00030827 ...,  0.00626651  0.01437919
   0.0079471 ]
 [ 0.00476062  0.00134965  0.00499129 ...,  0.00144392  0.01891362
   0.00282793]
 ..., 
 [ 0.00817678  0.00086976  0.00826191 ...,  0.00666214  0.01185722
   0.00572893]
 [ 0.02335731  0.00083561  0.00965767 ...,  0.00827377  0.00285444
   0.00861874]
 [ 0.03629782  0.01049384  0.00932367 ...

In [1]:
print("Soit {} mins".format(2748/60))


Soit 45.8 mins


In [2]:
print("Soit {} mins par octet de clé".format(46/16))

Soit 2.875 mins par octet de clé
